# Создание агента с функциями на базе GigaChat
## Инициализация GigaChat
Для работы агента нам понадобиться экземпляр класса GigaChat и доступ к модели, которая поддерживает работу с функциями. На данный момент не все модели поддерживают такой функционал.

In [9]:
from langchain.chat_models.gigachat import GigaChat

giga = GigaChat(credentials=..., model=...)

## Создание списка функций
Функции - это инструменты, доступные модели в прцоессе решения задачи. Модель сама принимает решение о том, когда и какой инструмент нужно использовать.
При обращении к инструменту он выполняется на на стороне клиента.

Для начала мы будем использовать только один инструмент - адаптер к поисковику [DuckDuckGo](https://www.duckduckgo.com).

In [10]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()
tools = [search_tool]

## Инициализация агента GigaChat
Теперь мы можем создать агента, который будет решать задачи с использованием доступных ему функций и срезу выполнения для этого агента.

In [11]:
from langchain.agents import AgentExecutor, create_gigachat_functions_agent

agent = create_gigachat_functions_agent(giga, tools)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
)

## Запуск агента

In [4]:
agent_executor.invoke(
    {"input": "Найди текущий курс биткоина и напечатай только число"}
)["output"]



> Entering new AgentExecutor chain...

Invoking: `duckduckgo_search` with `{'query': 'current bitcoin price'}`


Binance provides live Bitcoin price (BTC) in USD, with a current market cap of $882.53B USD and a 24-hour trading volume of $31.83B USD. Learn about Bitcoin's history, volatility, supply, and future halving event. The current price of Bitcoin is $46,652.86 per BTC, according to Coinbase. You can use the BTC to USD calculator to convert any amount of BTC to USD or vice versa. See the market cap, volume, circulating supply and historical data of Bitcoin and other crypto assets. Jan 23, 2024 - 7:46 NY Time Price 0.00 USD 0.00 (0.00%) Market Cap 0.00 Volume 0.00 The Kitco Bitcoin price index provides the latest Bitcoin price in US Dollars using an average from the world's leading exchanges. Get the latest Bitcoin price, live chart, and news on Blockchain.com, the world's most popular crypto wallet. Learn about Bitcoin's history, mining, supply, and features, and compare it wit

'46,652.86'

## Агент с несколькими функциями
Для демонстрации работы агентов, которые умеют использовать несколько инструментов, добавим ещё одну функцию - вывод крупного текста с использованием ASCII графики. Для этого нам понадобится библиотека pyfidget.

In [12]:
# %pip install pyfiglet
import pyfiglet

pyfiglet.print_figlet("Hello!", font="epic")

          _______  _        _        _______  _ 
|\     /|(  ____ \( \      ( \      (  ___  )( )
| )   ( || (    \/| (      | (      | (   ) || |
| (___) || (__    | |      | |      | |   | || |
|  ___  ||  __)   | |      | |      | |   | || |
| (   ) || (      | |      | |      | |   | |(_)
| )   ( || (____/\| (____/\| (____/\| (___) | _ 
|/     \|(_______/(_______/(_______/(_______)(_)
                                                



## Создание собственных функций
Для быстрого создания функций из python кода можно использовать декоратор @tool, который превращает любую функцию python в тул, доступный для вызова из модели.

> Обратите внимание! Модель будет ориентироваться на название, описание функции, а также описание и типы аргументов и возвращаемого значения. Если эти значения не указаны явно, то модель может "не понять", как следует использовать функцию.

In [13]:
from langchain.tools import tool


@tool
def draw_bunner(number: str) -> None:
    """Рисует баннер с текстом

    Args:
        number (str): Число, который нужно нарисовать на баннере
    """
    pyfiglet.print_figlet(number, font="epic")


new_tools = [search_tool, draw_bunner]
agent = create_gigachat_functions_agent(giga, new_tools)

Теперь запустим агента. Попросим его сначала найти нужное значение, а потом распечатать его в крупном формате.

In [64]:
agent_executor = AgentExecutor(
    agent=agent,
    tools=new_tools,
    verbose=True,
)

agent_executor.invoke(
    {"input": "Найди курс биткоина в долларах и нарисуй это число на банере."}
)["output"]



> Entering new AgentExecutor chain...

Invoking: `duckduckgo_search` with `{'query': 'курс биткоина в долларах'}`


Максимальная цена Биткоина - 68,770.00 USD (пик был достигнут 10 ноября 2021 года). ЧИТАЙТЕ: История Биткоина с Самого Начала: Создание криптовалют [2008-2021] и что влияло на цены исторически Довольно простой метод, который подойдет всем, и особенно новичкам - воспользоваться обменником " Матби ". 1 BTC = 41 575,74 USD по курсу на 21.01.2024. Стоимость 1 биткоина в долларах США на сегодня составляет 41 575,74 $ по данным ЦБ РФ, по сравнению со вчерашним днём курс валюты уменьшился на -0,42% (на -172,92 $). Курс биткоина по отношению к доллару США на графике, таблица динамики стоимости в процентах за день, неделю, месяц и год. Конвертер валют Узнайте курс биткоина в долларах в разных банках и обменниках, а также динамику их изменений за месяц и год. Сравните курсы биткоина в долларах с другими валютами и посмотрите средние курсы в банках, Межбанке, НБУ, Visa и Mastercar

''

# Добавляем память
Созданый нами агент не имеет состояния. Это означает, что он не помнит предыдущие взаимодействия. Чтобы предоставить ему память, нам нужно передать предыдущую `chat_history`. Примечание: имя `chat_history` используется, потому что оно явно задано в промпте. Если вы хотите использовать свой промпт, то сможете задать другое название переменной.

In [20]:
from langchain_core.messages import AIMessage, HumanMessage

agent_executor = AgentExecutor(
    agent=agent,
    tools=new_tools,
    verbose=False,
)

agent_executor.invoke(
    {
        "chat_history": [
            HumanMessage(
                content="Привет! Запомни трех животных - слон, жираф, крокодил"
            ),
            AIMessage(content="Привет! Хорошо, я запомнил."),
        ],
        "input": "Что я просил тебя запомнить?",
    }
)

{'chat_history': [HumanMessage(content='Привет! Запомни трех животных - слон, жираф, крокодил'),
  AIMessage(content='Привет! Хорошо, я запомнил.')],
 'input': 'Что я просил тебя запомнить?',
 'output': 'Вы просили меня запомнить трех животных: слон, жираф и крокодил.'}

## Чат с агентом
Вы можете реализовать взаимодействие агента и пользователя в режиме чата с сохранением истории общения и промежуточных результатов.

In [21]:
chat_history = []
while True:
    user_input = input("Вы: ")
    print(f"User: {user_input}")
    if user_input == "":
        break
    result = agent_executor.invoke(
        {
            "chat_history": chat_history,
            "input": user_input,
        }
    )
    chat_history.append(HumanMessage(content=user_input))
    chat_history.append(AIMessage(content=result["output"]))
    print(f"Bot: {result['output']}")

User: Привет
Bot: Здравствуйте! Могу ли я помочь Вам с чем-то?
User: Найди возраст Илона Маска
Bot: Илон Маск родился 28 июня 1971 года, поэтому на текущий момент (2022 год) ему 50 лет.
User: Выведи его возраст на банер
 _______  _______ 
(  ____ \(  __   )
| (    \/| (  )  |
| (____  | | /   |
(_____ \ | (/ /) |
      ) )|   / | |
/\____) )|  (__) |
\______/ (_______)
                  

Bot: Возраст Илона Маска - 50 лет.
User: 
